In [2]:
import json
import os
import glob

# Configuration
SOURCE_DIR = "individual_results"
FINAL_OUTPUT_DIR = "isingBatches"
FINAL_OUTPUT_FILENAME = "TSP_QAOA_IBM_batch_results_MERGED.json"

def merge_json_files():
    """
    Finds all individual result JSON files, combines them into a single
    structured file, and saves it.
    """
    all_results = []
    metadata = {}

    # Find all individual result files using a wildcard
    search_pattern = os.path.join(SOURCE_DIR, "result_instance_*.json")
    individual_files = glob.glob(search_pattern)

    if not individual_files:
        print(f"Error: No result files found in '{SOURCE_DIR}'.")
        return

    print(f"Found {len(individual_files)} result files to merge.")

    # Loop through each file, load its data, and append the result
    for file_path in individual_files:
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        # Add the main result data to our list
        all_results.append(data["result"])
        
        # Grab the metadata from the first file we process
        if not metadata:
            metadata = data["metadata"]

    # Optional: Sort the results by instance_id for a clean, ordered final file
    all_results.sort(key=lambda x: x['instance_id'])

    # Assemble the final data structure
    final_data = {
        "metadata": metadata,
        "results": all_results
    }

    # Ensure the final output directory exists
    os.makedirs(FINAL_OUTPUT_DIR, exist_ok=True)
    final_path = os.path.join(FINAL_OUTPUT_DIR, FINAL_OUTPUT_FILENAME)

    # Save the merged file
    with open(final_path, 'w') as f:
        # Use the regular json.dump since our custom encoder is not needed here
        json.dump(final_data, f, indent=4)

    print(f"Successfully merged {len(all_results)} results into {final_path}")

if __name__ == "__main__":
    merge_json_files()

Found 100 result files to merge.
Successfully merged 100 results into isingBatches/TSP_QAOA_IBM_batch_results_MERGED.json
